# 2023,2024 쿠팡 데이터로 2023 쿠팡 데이터를 예측해 유사성 측정 및 모델 추출

In [53]:
import pandas as pd

df = pd.read_excel('excel/coupang.xlsx')
df

,주문일,상품명,가격,카테고리
0,2025-01-15 02:19:03,자가비 케첩맛 3p,"7,180",음식 및 음료
1,2025-01-15 02:19:03,농심 육개장 사발면 86g,"4,980",음식 및 음료
2,2025-01-15 02:19:03,오뚜기 컵누들 마라탕 44.7g,"6,350",음식 및 음료
3,2025-01-15 02:19:03,라온 2in1 맥세이프 호환 듀얼 무선충전기 15W,"27,900",장비 및 생활용품
4,2025-01-13 05:06:48,솔향가득 잎편백수 천연 피톤치드 원액 스프레이 + 스프레이 공병,"16,500",장비 및 생활용품
...,...,...,...,...
821,2023-01-09 01:10:56,카누 다크로스트 아메리카노 원두커피믹스,"16,340",음식 및 음료
822,2023-01-08 03:56:31,고르고 바른 마스크 대형,"7,900",장비 및 생활용품
823,2023-01-05 02:42:00,공차 허니자몽블랙티,"7,380",음식 및 음료
824,2023-01-05 02:42:00,ipTIME USB허브 UC304,"14,240",장비 및 생활용품


In [54]:
df2023=pd.read_excel('excel/coupang2023.xlsx')

In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


# 라벨 인코딩
df['카테고리'] = df['카테고리'].astype('category')
label_mapping = {idx: label for idx, label in enumerate(df['카테고리'].cat.categories)}
reverse_label_mapping = {label: idx for idx, label in label_mapping.items()}
df['카테고리'] = df['카테고리'].cat.codes

print("Label Mapping:", label_mapping)
print("Reverse Label Mapping:", reverse_label_mapping)

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(df['상품명'], df['카테고리'], test_size=0.2, random_state=42)

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 로지스틱 회귀 모델 학습
model = LogisticRegression(max_iter=5000, random_state=42,class_weight='balanced')
model.fit(X_train_tfidf, y_train)

# 예측
y_pred = model.predict(X_test_tfidf)

# 성능 평가
print("\n모델 성능 평가:")
print(classification_report(y_test, y_pred, target_names=list(label_mapping.values()), zero_division=0))

# 새로운 데이터 예측
df_tfidf = vectorizer.transform(df2023['상품명'])
df2023['카테고리'] = model.predict(df_tfidf)
df2023['카테고리'] = df2023['카테고리'].map(label_mapping)

# 결과 확인
print("\n예측된 결과:")
print(df2023)


Label Mapping: {0: '음식 및 음료', 1: '장비 및 생활용품'}
Reverse Label Mapping: {'음식 및 음료': 0, '장비 및 생활용품': 1}

모델 성능 평가:
              precision    recall  f1-score   support

     음식 및 음료       0.91      0.99      0.95       108
   장비 및 생활용품       0.98      0.83      0.90        58

    accuracy                           0.93       166
   macro avg       0.95      0.91      0.92       166
weighted avg       0.94      0.93      0.93       166


예측된 결과:
                     주문일                          상품명      가격       카테고리
0    2023-12-21 08:09:34        켈로그 프로틴 그래놀라 쉐이크 미숫가루  19,900    음식 및 음료
1    2023-12-21 08:09:34                     쿠크다스 케이크  13,020    음식 및 음료
2    2023-12-21 08:09:34  미주라 통밀도넛츠 2p + 초코칩 통밀도넛츠 2p  17,500    음식 및 음료
3    2023-12-21 08:09:34               곰곰 간편한 미숫가루 스틱  16,300    음식 및 음료
4    2023-12-21 08:09:34               농심 짜파게티 범벅 70g   5,040    음식 및 음료
..                   ...                          ...     ...        ...
403  2023-01-09 01:10:56        카누 다크로스트 아

In [56]:
from sklearn.preprocessing import StandardScaler
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 로지스틱 회귀 모델 학습 및 GridSearchCV
param_grid = {
    'C': [0.1],
    'penalty': ['l2'],  # 정규화 방식
    'solver': ['liblinear', 'saga'],  # 최적화 알고리즘
    'class_weight': ['balanced']
}

grid = GridSearchCV(LogisticRegression(max_iter=5000, random_state=42), param_grid, cv=5, scoring='accuracy')
grid.fit(X_train_tfidf, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid.best_params_)

# 최적의 모델로 예측
y_pred = grid.best_estimator_.predict(X_test_tfidf)

# 성능 평가
print("\n모델 성능 평가:")
print(classification_report(y_test, y_pred, target_names=list(label_mapping.values()), zero_division=0))

Best Parameters: {'C': 0.1, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}

모델 성능 평가:
              precision    recall  f1-score   support

     음식 및 음료       0.92      1.00      0.96       108
   장비 및 생활용품       1.00      0.83      0.91        58

    accuracy                           0.94       166
   macro avg       0.96      0.91      0.93       166
weighted avg       0.94      0.94      0.94       166



In [57]:
# 새로운 데이터 예측 2023데이터터
df_tfidf = vectorizer.transform(df2023['상품명'])
df2023['카테고리'] = grid.best_estimator_.predict(df_tfidf)
df2023['카테고리'] = df2023['카테고리'].map(label_mapping)

# 결과 확인
print("\n예측된 결과:")
print(df2023)


예측된 결과:
                     주문일                          상품명      가격       카테고리
0    2023-12-21 08:09:34        켈로그 프로틴 그래놀라 쉐이크 미숫가루  19,900    음식 및 음료
1    2023-12-21 08:09:34                     쿠크다스 케이크  13,020    음식 및 음료
2    2023-12-21 08:09:34  미주라 통밀도넛츠 2p + 초코칩 통밀도넛츠 2p  17,500    음식 및 음료
3    2023-12-21 08:09:34               곰곰 간편한 미숫가루 스틱  16,300    음식 및 음료
4    2023-12-21 08:09:34               농심 짜파게티 범벅 70g   5,040    음식 및 음료
..                   ...                          ...     ...        ...
403  2023-01-09 01:10:56        카누 다크로스트 아메리카노 원두커피믹스  16,340    음식 및 음료
404  2023-01-08 03:56:31                고르고 바른 마스크 대형   7,900  장비 및 생활용품
405  2023-01-05 02:42:00                   공차 허니자몽블랙티   7,380    음식 및 음료
406  2023-01-05 02:42:00           ipTIME USB허브 UC304  14,240  장비 및 생활용품
407  2023-01-05 02:42:00        알티스트 핫초코 한잔에 당분이 단 1g   8,900    음식 및 음료

[408 rows x 4 columns]


In [58]:
df23=pd.read_excel('excel/coupang2023.xlsx') # 수작업라벨링링

In [59]:
import pandas as pd

# 두 데이터프레임의 길이 확인
if len(df23) != len(df2023):
    print("Warning: 두 데이터프레임의 길이가 다릅니다. 일치하는 인덱스만 비교됩니다.")

# 일치하는 인덱스 비교
comparison = df23['카테고리'] == df2023['카테고리']

# 결과 계산
total_count = len(comparison)  # 총 비교 개수
match_count = comparison.sum()  # 일치 개수
mismatch_count = total_count - match_count  # 불일치 개수
match_percentage = (match_count / total_count) * 100  # 일치 비율

# 결과 출력
print(f"총 개수: {total_count}")
print(f"일치 개수: {match_count}")
print(f"불일치 개수: {mismatch_count}")
print(f"일치율: {match_percentage:.2f}%")

총 개수: 408
일치 개수: 402
불일치 개수: 6
일치율: 98.53%


In [60]:
import pickle

# 모델 저장
with open('model/logistic_regression_model.pkl', 'wb') as file:
    pickle.dump(grid.best_estimator_, file)

# 벡터라이저 저장
with open('model/tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

print("모델과 벡터라이저가 성공적으로 저장되었습니다.")

모델과 벡터라이저가 성공적으로 저장되었습니다.


In [62]:
from joblib import dump, load
# 라벨 매핑 저장
dump(label_mapping, 'model/label_mapping.joblib')

['label_mapping.joblib']